# Near-Miss Incident Detection Pipeline
**Tatweer AI/ML Take-Home Challenge — Option 1: Computer Vision**

This notebook implements a full end-to-end system for detecting near-miss traffic incidents in a video clip.  
All core logic lives in `src/` Python modules; this notebook orchestrates and visualises results.

**Pipeline overview:**
1. Download & inspect the video
2. Detect objects with YOLOv8n (CPU)
3. Track objects with a Centroid Tracker
4. Detect near-miss events (proximity + TTC + risk scoring)
5. Generate annotated video + dashboard visualisations + HTML report
6. **Bonus A** — False-positive filter comparison
7. **Bonus B** — Real-time performance benchmark

---
## 1. Setup
Install dependencies and configure paths.

In [ ]:
# ── Install dependencies (safe to re-run) ─────────────────────────────────
import subprocess, sys

pkgs = [
    'opencv-python',
    'ultralytics',
    'yt-dlp',
    'numpy',
    'matplotlib',
    'seaborn',
    'pandas',
    'scipy',
    'Pillow',
    'onnxruntime',
]

# CPU-only torch to keep install fast on Colab
torch_cmd = [
    sys.executable, '-m', 'pip', 'install', '-q',
    'torch', 'torchvision',
    '--index-url', 'https://download.pytorch.org/whl/cpu'
]

subprocess.run([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs, check=True)
subprocess.run(torch_cmd, check=True)

print('All packages installed.')

In [3]:
import os, sys

# ── Resolve project root (works on Colab and locally) ─────────────────────
NOTEBOOK_DIR = os.path.abspath('.')
# If running from notebooks/, go up one level
PROJECT_ROOT = (
    os.path.dirname(NOTEBOOK_DIR)
    if os.path.basename(NOTEBOOK_DIR) == 'notebooks'
    else NOTEBOOK_DIR
)
SRC_DIR = os.path.join(PROJECT_ROOT, 'src')
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'outputs')

for d in [SRC_DIR, DATA_DIR, OUTPUT_DIR]:
    os.makedirs(d, exist_ok=True)

if SRC_DIR not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f'Project root : {PROJECT_ROOT}')
print(f'Source dir   : {SRC_DIR}')
print(f'Output dir   : {OUTPUT_DIR}')

Project root : /Users/mina.essam/Downloads/tatweer
Source dir   : /Users/mina.essam/Downloads/tatweer/src
Output dir   : /Users/mina.essam/Downloads/tatweer/outputs


In [4]:
import math
import time
from collections import defaultdict

import cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from IPython.display import display, HTML, Image as IPImage

from src.video_utils import (
    download_video, get_video_metadata, sample_frames,
    frame_generator, display_sample_frames,
)
from src.detector import ObjectDetector, draw_detections
from src.tracker import create_tracker
from src.near_miss import NearMissDetectorV11
from src.visualizer import (
    annotate_frame, create_annotated_video,
    plot_timeline, plot_risk_distribution,
    plot_heatmap, plot_frequency,
    generate_html_report,
)

print('Imports OK.')

Imports OK.


---
## 2. Video Download & Metadata

In [6]:
VIDEO_URL = 'https://www.youtube.com/watch?v=r86kxxU-LUY'
VIDEO_URL = 'https://www.youtube.com/watch?v=YF_DzoTDO-0'
VIDEO_PATH = os.path.join(DATA_DIR, 'traffic_video_3.mp4')

if not os.path.exists(VIDEO_PATH):
    print('Downloading video...')
    download_video(VIDEO_URL, VIDEO_PATH, quality='720')
else:
    print(f'Video already exists: {VIDEO_PATH}')

meta = get_video_metadata(VIDEO_PATH)
print('\n── Video Metadata ────────────────────────')
for k, v in meta.items():
    if k == 'duration_sec':
        mm = int(v // 60); ss = v % 60
        print(f'  {k:15s}: {mm:02d}:{ss:05.2f} ({v:.1f}s)')
    else:
        print(f'  {k:15s}: {v}')

[youtube] Extracting URL: https://www.youtube.com/watch?v=YF_DzoTDO-0
[youtube] YF_DzoTDO-0: Downloading webpage


[youtube] YF_DzoTDO-0: Downloading android vr player API JSON
[info] YF_DzoTDO-0: Downloading 1 format(s): 398
[download] Destination: /Users/mina.essam/Downloads/tatweer/data/traffic_video_3.mp4
[download] 100% of    4.05MiB in 00:00:22 at 180.79KiB/s 

── Video Metadata ────────────────────────
  fps            : 30.0
  frame_count    : 888
  width          : 1280
  height         : 720
  duration_sec   : 00:29.60 (29.6s)


In [ ]:
# Display 5 evenly-spaced frames to visually inspect the video
frames = sample_frames(VIDEO_PATH, n=5)

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for ax, (idx, frame) in zip(axes, frames):
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    ax.imshow(rgb)
    ts = idx / meta['fps']
    ax.set_title(f'Frame {idx}\n{ts:.1f}s', fontsize=9)
    ax.axis('off')
plt.suptitle('Sample Frames', fontsize=13)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'sample_frames.png'), dpi=100, bbox_inches='tight')
plt.show()
print('Sample frames saved.')

---
## 3. Object Detection Preview
Run YOLOv8n on 10 sampled frames and display annotated results.

In [ ]:
# Load detector — downloads yolov8n.pt on first run (~6 MB)
FRAME_STRIDE = 2        # process every 2nd frame
INPUT_SIZE   = 640      # YOLO input resolution
CONF_THRESH  = 0.40

detector = ObjectDetector(
    model_name='yolov8n.pt',
    conf=CONF_THRESH,
    input_size=INPUT_SIZE,
)
print('Detector ready.')

In [ ]:
# Run detection on 10 sample frames, display annotated grid
preview_frames = sample_frames(VIDEO_PATH, n=10)

fig, axes = plt.subplots(2, 5, figsize=(22, 8))
axes = axes.flatten()

detection_counts = []
for ax, (idx, frame) in zip(axes, preview_frames):
    dets = detector.detect(frame)
    detection_counts.append(len(dets))
    annotated = draw_detections(frame, dets)
    rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    ax.imshow(rgb)
    ts = idx / meta['fps']
    ax.set_title(f'Frame {idx} ({ts:.1f}s)\n{len(dets)} detections', fontsize=8)
    ax.axis('off')

plt.suptitle('Detection Preview — YOLOv8n', fontsize=13)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'detection_preview.png'), dpi=100, bbox_inches='tight')
plt.show()

print(f'\nAvg inference time : {detector.avg_inference_ms:.1f} ms/frame')
print(f'Avg detections     : {sum(detection_counts)/len(detection_counts):.1f} per frame')
detector.reset_timing()

---
## 4. Full Pipeline Run
Process the entire video: detect → track → near-miss detection.

In [ ]:
# Pipeline configuration
EFFECTIVE_FPS   = meta['fps'] / FRAME_STRIDE   # FPS after stride
PROXIMITY_PX    = 120          # centroid distance threshold (px)
TTC_THRESHOLD   = 2.5          # time-to-collision threshold (seconds)
DEBOUNCE_FRAMES = 30           # min frames between same-pair events
MAX_DISTANCE    = 200          # max tracking match distance (px)
MAX_DISAPPEARED = 15           # frames before deregistering a track

print(f'Effective FPS      : {EFFECTIVE_FPS:.2f}')
print(f'Proximity threshold: {PROXIMITY_PX} px')
print(f'TTC threshold      : {TTC_THRESHOLD} s')

In [ ]:
tracker = create_tracker(
    'centroid',
    max_disappeared=MAX_DISAPPEARED,
    max_distance=MAX_DISTANCE,
    min_hits=2,
)
nm_detector = NearMissDetectorV11(
    proximity_px=PROXIMITY_PX,
    ttc_threshold=TTC_THRESHOLD,
    fps=EFFECTIVE_FPS,
    debounce_frames=DEBOUNCE_FRAMES,
    filters_enabled=True,

    clearance_scale=1.1,

)

# frame_data[frame_idx] = (tracked_objects_snapshot, active_pairs)
frame_data  = {}
peak_active = 0

t_start = time.perf_counter()

for frame_idx, frame in frame_generator(VIDEO_PATH, stride=FRAME_STRIDE):
    # 1. Detect
    dets = detector.detect(frame)

    # 2. Track
    tracked = tracker.update(dets, frame_idx)
    peak_active = max(peak_active, len(tracked))

    # 3. Near-miss
    events = nm_detector.process_frame(frame_idx, tracked)
    active_pairs = nm_detector.active_pairs(frame_idx)

    # Save snapshot for annotation pass
    frame_data[frame_idx] = (dict(tracked), list(active_pairs))

t_end = time.perf_counter()
total_time = t_end - t_start

events_df = nm_detector.get_events_dataframe()
summary   = nm_detector.summary()

print(f'\n── Pipeline Complete ──────────────────────────────')
print(f'  Total processing time : {total_time/60:.1f} min ({total_time:.0f}s)')
print(f'  Avg inference         : {detector.avg_inference_ms:.1f} ms/frame')
print(f'  Frames processed      : {len(frame_data)}')
print(f'  Unique object IDs     : {tracker.total_ids_assigned}')
print(f'  Peak simultaneous     : {peak_active} objects')
print(f'  Near-miss events      : {summary["total"]} total')
print(f'    High  : {summary["high"]}')
print(f'    Medium: {summary["medium"]}')
print(f'    Low   : {summary["low"]}')

---
## 5. Event Results

In [ ]:
if events_df.empty:
    print('No near-miss events detected. Try lowering PROXIMITY_PX or TTC_THRESHOLD.')
else:
    print(f'Total events: {len(events_df)}')
    display(events_df.head(20))

In [ ]:
if not events_df.empty:
    print('── Top 5 Highest-Risk Events ──────────────')
    top5 = events_df.nlargest(5, 'risk_score')[
        ['timestamp_sec', 'class_1', 'class_2',
         'distance_px', 'ttc_sec', 'risk_score', 'risk_level']
    ]
    display(top5)

---
## 6. Dashboard Visualizations

In [ ]:
# Generate all 4 charts
path_timeline = os.path.join(OUTPUT_DIR, 'timeline.png')
path_pie      = os.path.join(OUTPUT_DIR, 'risk_distribution.png')
path_heatmap  = os.path.join(OUTPUT_DIR, 'heatmap.png')
path_freq     = os.path.join(OUTPUT_DIR, 'frequency.png')

# First frame for heatmap background
cap = cv2.VideoCapture(VIDEO_PATH)
_, first_frame = cap.read()
cap.release()

plot_timeline(events_df, meta['duration_sec'], path_timeline)
plot_risk_distribution(events_df, path_pie)
plot_heatmap(tracker.all_trajectories, first_frame, path_heatmap)
plot_frequency(events_df, meta['duration_sec'], path_freq, bin_sec=15)

# Display all inline
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
chart_paths = [
    (path_timeline, 'Event Timeline'),
    (path_pie, 'Risk Distribution'),
    (path_heatmap, 'Activity Heatmap'),
    (path_freq, 'Frequency Over Time'),
]
for ax, (path, title) in zip(axes.flatten(), chart_paths):
    if os.path.exists(path):
        img = mpimg.imread(path)
        ax.imshow(img)
        ax.set_title(title, fontsize=12)
    ax.axis('off')

plt.suptitle('Near-Miss Detection Dashboard', fontsize=15, y=1.01)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'dashboard.png'), dpi=120, bbox_inches='tight')
plt.show()
print('All charts saved to outputs/.')

---
## 7. Annotated Output Video

In [ ]:
VIDEO_OUT = os.path.join(OUTPUT_DIR, 'annotated_video.mp4')

print('Writing annotated video (this may take a few minutes)...')
create_annotated_video(
    video_path=VIDEO_PATH,
    output_path=VIDEO_OUT,
    frame_data=frame_data,
    fps=EFFECTIVE_FPS,
)

size_mb = os.path.getsize(VIDEO_OUT) / 1e6
print(f'Done. File size: {size_mb:.1f} MB')

# On Colab, provide a download link
try:
    from google.colab import files
    print('Downloading annotated_video.mp4 to your machine...')
    files.download(VIDEO_OUT)
except ImportError:
    print(f'Output saved locally at: {VIDEO_OUT}')

---
## 8. HTML Summary Report

In [ ]:
REPORT_PATH = os.path.join(OUTPUT_DIR, 'report.html')

# Compute tracker stats for the report
all_trajs = tracker.all_trajectories
avg_track_len = (
    round(sum(len(t) for t in all_trajs.values()) / len(all_trajs), 1)
    if all_trajs else 0
)
tracker_stats = {
    'total_ids': tracker.total_ids_assigned,
    'avg_track_length': avg_track_len,
    'peak_active': peak_active,
}

img_paths = {
    'timeline': path_timeline,
    'risk_distribution': path_pie,
    'heatmap': path_heatmap,
    'frequency': path_freq,
}

generate_html_report(
    metadata=meta,
    events_df=events_df,
    tracker_stats=tracker_stats,
    img_paths=img_paths,
    output_path=REPORT_PATH,
)

# On Colab, provide a download link
try:
    from google.colab import files
    files.download(REPORT_PATH)
except ImportError:
    print(f'Report saved locally at: {REPORT_PATH}')

---
## 9. Bonus A — False-Positive Filter Comparison
Re-run near-miss detection **without** the FP filters and compare event counts.

In [ ]:
# Re-run with filters DISABLED (reuse the already-computed tracker.all_trajectories)
nm_unfiltered = NearMissDetectorV11(
    proximity_px=PROXIMITY_PX,
    ttc_threshold=TTC_THRESHOLD,
    fps=EFFECTIVE_FPS,
    debounce_frames=DEBOUNCE_FRAMES,
    filters_enabled=False,   # <-- key difference
    clearance_scale=1.1,
)

for frame_idx, (tracked_snap, _) in frame_data.items():
    nm_unfiltered.process_frame(frame_idx, tracked_snap)

unfiltered_df = nm_unfiltered.get_events_dataframe()
filtered_df   = events_df  # already computed with filters=True

# Comparison table
def summarise(df, label):
    if df.empty:
        return {'Config': label, 'Total': 0, 'High': 0, 'Medium': 0, 'Low': 0}
    counts = df['risk_level'].value_counts().to_dict()
    return {
        'Config': label,
        'Total': len(df),
        'High': counts.get('High', 0),
        'Medium': counts.get('Medium', 0),
        'Low': counts.get('Low', 0),
    }

cmp_df = pd.DataFrame([
    summarise(unfiltered_df, 'Without FP Filters'),
    summarise(filtered_df,   'With FP Filters'),
])
cmp_df['Reduction %'] = [
    0.0,
    round((1 - len(filtered_df) / max(len(unfiltered_df), 1)) * 100, 1),
]

print('── False-Positive Filter Comparison ──────────')
display(cmp_df)

# Bar chart
fig, ax = plt.subplots(figsize=(7, 4))
x = range(len(cmp_df))
ax.bar([i - 0.2 for i in x], cmp_df['High'],   0.18, label='High',   color='#e74c3c')
ax.bar([i       for i in x], cmp_df['Medium'], 0.18, label='Medium', color='#e67e22')
ax.bar([i + 0.2 for i in x], cmp_df['Low'],    0.18, label='Low',    color='#f1c40f')
ax.set_xticks(list(x))
ax.set_xticklabels(cmp_df['Config'])
ax.set_ylabel('Event Count')
ax.set_title('Effect of False-Positive Filters')
ax.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'fp_filter_comparison.png'), dpi=120, bbox_inches='tight')
plt.show()
print('Comparison chart saved.')

### Manual Event Review
Display the 5 events that were **filtered out** (in unfiltered but not in filtered).

In [ ]:
# Events present in unfiltered but not in filtered (by frame_index + id pair)
def event_key(row):
    return (row['frame_index'], min(row['object_id_1'], row['object_id_2']),
            max(row['object_id_1'], row['object_id_2']))

if not unfiltered_df.empty and not filtered_df.empty:
    unf_keys = set(unfiltered_df.apply(event_key, axis=1))
    fil_keys  = set(filtered_df.apply(event_key, axis=1))
    removed_keys = unf_keys - fil_keys

    removed_events = unfiltered_df[
        unfiltered_df.apply(event_key, axis=1).isin(removed_keys)
    ].head(5)

    if removed_events.empty:
        print('No events were filtered out — filters had no effect on this video.')
    else:
        print(f'Showing {len(removed_events)} filtered-out event(s):')
        display(removed_events[['frame_index','timestamp_sec','class_1','class_2',
                                 'distance_px','ttc_sec','risk_score','risk_level']])
elif unfiltered_df.empty:
    print('Unfiltered detector found no events either.')
else:
    print('All events were retained by the filter.')

---
## 10. Bonus B — Performance Benchmark
Compare FPS across different stride/resolution/backend configurations.

In [ ]:
from src.optimizer import benchmark_pipeline, recommend_config, export_to_onnx

# Export ONNX model for benchmarking
ONNX_PATH = os.path.join(OUTPUT_DIR, 'yolov8n.onnx')
if not os.path.exists(ONNX_PATH):
    print('Exporting YOLOv8n to ONNX...')
    exported = export_to_onnx(detector.model, ONNX_PATH)
    # ultralytics saves to the model's location; copy to OUTPUT_DIR if needed
    import shutil
    if not os.path.exists(ONNX_PATH) and os.path.exists('yolov8n.onnx'):
        shutil.move('yolov8n.onnx', ONNX_PATH)
else:
    print(f'ONNX model already exists: {ONNX_PATH}')

# Benchmark configs
configs = [
    {'stride': 1, 'input_size': 640, 'backend': 'pytorch'},
    {'stride': 2, 'input_size': 640, 'backend': 'pytorch'},
    {'stride': 4, 'input_size': 640, 'backend': 'pytorch'},
    {'stride': 2, 'input_size': 320, 'backend': 'pytorch'},
    {'stride': 2, 'input_size': 640, 'backend': 'onnx', 'onnx_path': ONNX_PATH},
    {'stride': 2, 'input_size': 320, 'backend': 'onnx', 'onnx_path': ONNX_PATH},
]

bench_df = benchmark_pipeline(VIDEO_PATH, configs, n_frames=60)
print('\n── Benchmark Results ─────────────────────────────')
display(bench_df)

best = recommend_config(bench_df)
print(f'\nRecommended config: {best}')

In [ ]:
# Visualise benchmark results
fig, ax = plt.subplots(figsize=(10, 4))
colors = ['#e74c3c' if fps >= 15 else '#3498db' for fps in bench_df['fps']]
bars = ax.barh(bench_df['config_name'], bench_df['fps'], color=colors)
ax.axvline(x=15, color='red', linestyle='--', linewidth=1.5, label='15 FPS target')
ax.set_xlabel('Throughput (FPS)')
ax.set_title('Pipeline Benchmark — CPU Throughput')
ax.legend()

for bar, fps in zip(bars, bench_df['fps']):
    ax.text(bar.get_width() + 0.2, bar.get_y() + bar.get_height() / 2,
            f'{fps:.1f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'benchmark.png'), dpi=120, bbox_inches='tight')
plt.show()
print('Benchmark chart saved.')

### Benchmark Analysis

**Key trade-offs observed:**

| Optimization | FPS Impact | Quality Impact |
|---|---|---|
| Stride 1 → 4 | ~4× faster | May miss short-duration events |
| Resolution 640 → 320 | ~2× faster | Lower detection accuracy for small/distant objects |
| PyTorch → ONNX | ~1.2–1.5× faster | Same accuracy (identical weights) |

**Production recommendation:**  
For a real deployment with fixed cameras, `stride=2 + input_size=320 + ONNX` offers the best  
throughput/quality balance. For archival analysis where time is not critical, `stride=1 + 640px` 
gives the most accurate near-miss detection.

---
## Output Files Summary

| File | Description |
|---|---|
| `outputs/annotated_video.mp4` | Full video with bounding boxes and risk overlays |
| `outputs/report.html` | Self-contained HTML report with embedded charts |
| `outputs/timeline.png` | Near-miss events on the video timeline |
| `outputs/risk_distribution.png` | Pie chart of High / Medium / Low events |
| `outputs/heatmap.png` | Object activity density heatmap |
| `outputs/frequency.png` | Event frequency per 15-second interval |
| `outputs/fp_filter_comparison.png` | Before/after false-positive filtering |
| `outputs/benchmark.png` | CPU throughput across configurations |
| `outputs/dashboard.png` | Combined 4-chart dashboard |
| `outputs/detection_preview.png` | Sample detection frames grid |